In [2]:
from google.colab import drive
drive.mount('/content/drive')
SAVE_DIR = '/content/drive/MyDrive/Colab_Notebooks2/LLM_files'

Mounted at /content/drive


In [ ]:
# Google Colab Setup Script for LLM + FA Detection EDA Flow

# 1. System update & dependencies
!apt-get update -qq && apt-get install -y build-essential clang bison flex libreadline-dev \
  gawk tcl-dev libffi-dev git mercurial graphviz xdot pkg-config python3 python3-pip \
  libboost-system-dev libboost-python-dev libboost-filesystem-dev zlib1g-dev

# 2. Clone and build Yosys
!git clone https://github.com/YosysHQ/yosys.git
%cd yosys
!git submodule update --init --recursive
!make -j$(nproc)
!make install
!cp -r /content/yosys SAVE_DIR



In [ ]:
!cp -r /content/yosys /content/drive/MyDrive/Colab_Notebooks2/LLM_files/

In [3]:
SAVE_DIR

'/content/drive/MyDrive/Colab_Notebooks2/LLM_files'

In [88]:
# 3. Go back and create project directories
import os
%cd /
verilog_path = os.path.join(SAVE_DIR, 'llm_fa/verilog_src')
!mkdir -p /content/llm_fa/verilog_src /content/llm_fa/scripts /content/llm_fa/lib

# 4. Create example FA Verilog
fa_verilog = '''
module top(input a, b, cin, output sum, carry);
  assign sum = a ^ b ^ cin;
  assign carry = (a & b) | (b & cin) | (a & cin);
endmodule
'''
with open("/content/llm_fa/verilog_src/fa_example.v", "w") as f:
    f.write(fa_verilog)

# 5. Create FA cell primitive
fa_cell = '''
module FA_CELL(input a, b, cin, output sum, cout);
  assign sum = a ^ b ^ cin;
  assign cout = (a & b) | (cin & (a ^ b));
endmodule
'''
with open("/content/llm_fa/verilog_src/fa_cell.v", "w") as f:
    f.write(fa_cell)

# 6. Create yosys script to generate AIG
synth_script = '''
read_verilog /content/llm_fa/verilog_src/fa_example.v
hierarchy -top top
proc; opt; techmap
aigmap
write_aiger -ascii -symbols /content/llm_fa/design.aag
'''
#write_aiger /content/llm_fa/design.aig
with open("/content/llm_fa/scripts/synth.ys", "w") as f:
    f.write(synth_script)

print("✅ Setup complete. You can now run Yosys on /content/llm_fa/scripts/synth.ys")

/
✅ Setup complete. You can now run Yosys on /content/llm_fa/scripts/synth.ys


In [89]:
# 7. Run Yosys synthesis to generate AIG
%cd /content/drive/MyDrive/Colab_Notebooks2/LLM_files/yosys
!chmod +x yosys
!./yosys -V
!./yosys -s /content/llm_fa/scripts/synth.ys


/content/drive/MyDrive/Colab_Notebooks2/LLM_files/yosys
Yosys 0.53+101 (git sha1 0b19f628e, g++ 11.4.0-1ubuntu1~22.04 -fPIC -O3)

 /----------------------------------------------------------------------------\
 |  yosys -- Yosys Open SYnthesis Suite                                       |
 |  Copyright (C) 2012 - 2025  Claire Xenia Wolf <claire@yosyshq.com>         |
 |  Distributed under an ISC-like license, type "license" to see terms        |
 \----------------------------------------------------------------------------/
 Yosys 0.53+101 (git sha1 0b19f628e, g++ 11.4.0-1ubuntu1~22.04 -fPIC -O3)

-- Executing script file `/content/llm_fa/scripts/synth.ys' --

1. Executing Verilog-2005 frontend: /content/llm_fa/verilog_src/fa_example.v
Parsing Verilog input from `/content/llm_fa/verilog_src/fa_example.v' to AST representation.
Generating RTLIL representation for module `\top'.
Successfully finished Verilog frontend.

2. Executing HIERARCHY pass (managing design hierarchy).

2.1. Analyz

In [90]:
# 8. Rule-based FA pattern detection from AAG
def parse_aag_and_extract_fa(aag_path):
    fa_matches = []
    with open(aag_path) as f:
        lines = f.readlines()

    header = lines[0].strip().split()
    if header[0] != "aag":
        raise ValueError("Not a valid AAG file")

    M, I, L, O, A = map(int, header[1:])

    inputs = lines[1:I+1]
    outputs = lines[I+1:I+1+O]
    ands = lines[I+1+O:I+1+O+A]

    literal_to_inputs = {}

    for line in ands:
        parts = list(map(int, line.strip().split()))
        out, in0, in1 = parts
        literal_to_inputs[out] = (in0, in1)

    # 룰 기반: 두 AND 게이트가 3개의 입력을 공유하면 FA 후보
    keys = list(literal_to_inputs.keys())
    for i in range(len(keys)):
        for j in range(i + 1, len(keys)):
            out1, out2 = keys[i], keys[j]
            in_set = {
                literal_to_inputs[out1][0], literal_to_inputs[out1][1],
                literal_to_inputs[out2][0], literal_to_inputs[out2][1]
            }
            node_ids = {x // 2 for x in in_set}
            if len(node_ids) == 3:
                fa_matches.append({
                    "sum_node": out1,
                    "carry_node": out2,
                    "inputs": sorted(list(node_ids))
                })

    return fa_matches
def get_symbol_map(aag_path):
    symbol_map = {}
    with open(aag_path) as f:
        for line in f:
            if line.startswith(('i', 'o')) and ' ' in line:
                key, name = line.strip().split()
                index = int(key[1:])  # 'i0' → 0
                if key.startswith('i'):
                    literal = 2 * (index + 1)  # AIG literal: i0 → 2, i1 → 4 ...
                elif key.startswith('o'):
                    # output literals는 위에서 따로 읽어야 정확
                    continue
                symbol_map[literal] = name
    return symbol_map
def add_names_to_fa_matches(fa_matches, symbol_map):
    for fa in fa_matches:
        fa["input_names"] = [symbol_map.get(lit * 2, f"n{lit}") for lit in fa["inputs"]]
    return fa_matches

matches = parse_aag_and_extract_fa("/content/llm_fa/design.aag")
symbol_map = get_symbol_map("/content/llm_fa/design.aag")
fa_matches = add_names_to_fa_matches(matches, symbol_map)
#print(json.dumps(matches, indent=2))
formatted_fa = "\n".join([
    f"- Inputs: {', '.join(fa['input_names'])}\n  sum_node: {fa['sum_node']}\n  carry_node: {fa['carry_node']}"
    for fa in fa_matches
])
print(formatted_fa)

- Inputs: a, b, cin
  sum_node: 8
  carry_node: 20
- Inputs: a, b, cin
  sum_node: 8
  carry_node: 22
- Inputs: a, b, cin
  sum_node: 10
  carry_node: 20
- Inputs: a, b, cin
  sum_node: 10
  carry_node: 22
- Inputs: a, cin, n6
  sum_node: 14
  carry_node: 20
- Inputs: b, cin, n6
  sum_node: 14
  carry_node: 22
- Inputs: a, cin, n6
  sum_node: 16
  carry_node: 20
- Inputs: b, cin, n6
  sum_node: 16
  carry_node: 22
- Inputs: a, b, cin
  sum_node: 20
  carry_node: 22
- Inputs: a, b, cin
  sum_node: 20
  carry_node: 24
- Inputs: a, b, cin
  sum_node: 22
  carry_node: 24


In [91]:
# 9. Generate LLM prompt to locate FA logic in Verilog
prompt_template = f"""
You are given a Verilog source file and an AIG-level detection result showing the following FA-related outputs:

{formatted_fa}

Here is the Verilog source:

{fa_verilog}

Please identify the line(s) that implement a Full Adder and annotate them with a comment like:
// FA DETECTED START
...
// FA DETECTED END
"""

with open("/content/llm_fa/llm_prompt.txt", "w") as f:
    f.write(prompt_template)

print("🧠 LLM prompt generated. Ready to pass to a model like GPT for annotation.")

🧠 LLM prompt generated. Ready to pass to a model like GPT for annotation.


In [92]:
import os
from openai import OpenAI
client = OpenAI()

os.environ["OPENAI_API_KEY"] = "sk-...{user-api-key}"
openai.api_key = os.environ.get("OPENAI_API_KEY")
# models = client.models.list()
# for m in models.data:
#     print(m.id)

# 10. Call LLM to annotate Verilog code
with open("/content/llm_fa/llm_prompt.txt", "r") as f:
    prompt = f.read()

response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)
verilog_labeled = response.choices[0].message.content
with open("/content/llm_fa/verilog_src/fa_example_mod.v", "w") as f:
    f.write(verilog_labeled)

print("📝 LLM-labeled Verilog code saved to fa_example_mod.v")

Given the **AIG-level detection results**, it is clear the tool has detected the canonical Full Adder implementation:  
- **Inputs:** `a, b, cin`
- **Outputs:** `sum = a ^ b ^ cin`, `carry = (a & b) | (b & cin) | (a & cin)`

This matches exactly with the two assignment statements in your Verilog module.

### Annotated Verilog Source

```verilog
module top(input a, b, cin, output sum, carry);
  // FA DETECTED START
  assign sum = a ^ b ^ cin;
  assign carry = (a & b) | (b & cin) | (a & cin);
  // FA DETECTED END
endmodule
```

**Both assignment lines correspond to the Full Adder functionality as detected by the tool.**
📝 LLM-labeled Verilog code saved to fa_example_mod.v


In [93]:
# # 10. Replace detected logic with FA_CELL manually (simulated LLM behavior)
fa_modified = '''
module top(input a, b, cin, output sum, carry);
  // FA DETECTED START
  FA_CELL fa1 (.a(a), .b(b), .cin(cin), .sum(sum), .cout(carry));
  // FA DETECTED END
endmodule
'''
with open("/content/llm_fa/verilog_src/fa_example_mod.v", "w") as f:
    f.write(fa_modified)

# 11. Re-synthesize modified Verilog
synth_mod_script = '''
read_verilog /content/llm_fa/verilog_src/fa_cell.v
read_verilog /content/llm_fa/verilog_src/fa_example_mod.v
hierarchy -top top
proc; opt; techmap
stat
write_verilog /content/llm_fa/verilog_src/netlist_mod.v
'''
with open("/content/llm_fa/scripts/synth_mod.ys", "w") as f:
    f.write(synth_mod_script)

!./yosys -s /content/llm_fa/scripts/synth_mod.ys
print("✅ Re-synthesis complete with FA_CELL instantiated.")



 /----------------------------------------------------------------------------\
 |  yosys -- Yosys Open SYnthesis Suite                                       |
 |  Copyright (C) 2012 - 2025  Claire Xenia Wolf <claire@yosyshq.com>         |
 |  Distributed under an ISC-like license, type "license" to see terms        |
 \----------------------------------------------------------------------------/
 Yosys 0.53+101 (git sha1 0b19f628e, g++ 11.4.0-1ubuntu1~22.04 -fPIC -O3)

-- Executing script file `/content/llm_fa/scripts/synth_mod.ys' --

1. Executing Verilog-2005 frontend: /content/llm_fa/verilog_src/fa_cell.v
Parsing Verilog input from `/content/llm_fa/verilog_src/fa_cell.v' to AST representation.
Generating RTLIL representation for module `\FA_CELL'.
Successfully finished Verilog frontend.

2. Executing Verilog-2005 frontend: /content/llm_fa/verilog_src/fa_example_mod.v
Parsing Verilog input from `/content/llm_fa/verilog_src/fa_example_mod.v' to AST representation.
Generating RTLIL r

In [94]:
# 12. Compare before and after synthesis results (gate count)
original_stat_script = '''
read_verilog /content/llm_fa/verilog_src/fa_example.v
hierarchy -top top
proc; opt; techmap
stat
'''
with open("/content/llm_fa/scripts/stat_orig.ys", "w") as f:
    f.write(original_stat_script)

mod_stat_script = '''
read_verilog /content/llm_fa/verilog_src/fa_cell.v
read_verilog /content/llm_fa/verilog_src/fa_example_mod.v
hierarchy -top top
proc; opt; techmap
stat
'''
with open("/content/llm_fa/scripts/stat_mod.ys", "w") as f:
    f.write(mod_stat_script)


print("\n📊 Original design stats saved at '/content/llm_fa/stats/stat_orig.txt'")
os.makedirs("/content/llm_fa/stats", exist_ok=True)

!./yosys -s /content/llm_fa/scripts/stat_orig.ys > /content/llm_fa/stats/stat_orig.txt
print("\n📊 Modified design stats with FA_CELL saved at '/content/llm_fa/stats/stat_mod.txt'")
!./yosys -s /content/llm_fa/scripts/stat_mod.ys   > /content/llm_fa/stats/stat_mod.txt



📊 Original design stats saved at '/content/llm_fa/stats/stat_orig.txt'

📊 Modified design stats with FA_CELL saved at '/content/llm_fa/stats/stat_mod.txt'


In [95]:
def print_gate_summary(stat_path):
    important_cells = ["FA_CELL", "$_AND_", "$_OR_", "$_XOR_"]
    found = {cell: 0 for cell in important_cells}

    with open(stat_path, "r") as f:
        for line in f:
            for cell in important_cells:
                if line.strip().startswith(cell):
                    count = int(line.strip().split()[-1])
                    found[cell] = count

    for cell in important_cells:
        print(f"{cell:>8}: {found[cell]}")
print("📊 Original design summary:")
print_gate_summary("/content/llm_fa/stats/stat_orig.txt")
print("\n📊 Modified design summary")
print_gate_summary("/content/llm_fa/stats/stat_mod.txt")

📊 Original design summary:
 FA_CELL: 0
  $_AND_: 3
   $_OR_: 2
  $_XOR_: 2

📊 Modified design summary
 FA_CELL: 1
  $_AND_: 2
   $_OR_: 1
  $_XOR_: 2
